Video Classification Model : https://towardsdatascience.com/transfer-learning-using-mobilenet-and-keras-c75daf7ff299

## STEP 1:  import the required libraries

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
from glob import glob
import os

from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import imagenet_utils
from keras.applications import MobileNet
from keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


## STEP 2: Read each frame and their corresponding tag

In [2]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,A_OffSide_shot1.mp4_frame100.jpg,OffSide
1,A_OffSide_shot1.mp4_frame101.jpg,OffSide
2,A_OffSide_shot1.mp4_frame102.jpg,OffSide
3,A_OffSide_shot1.mp4_frame103.jpg,OffSide
4,A_OffSide_shot1.mp4_frame104.jpg,OffSide


## STEP 3: Read the frames and then store them as a NumPy array

In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(299,299,))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2596/2596 [00:38<00:00, 67.47it/s]


(2596, 299, 299, 3)

## STEP 4: Create the validation set

In [4]:
# separating the target
y = train['class']

In [5]:
# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

## STEP 5: Create 4 different columns in the target, one for each category

In [6]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [7]:
train['class'].value_counts()

OffSide    1756
Legside     840
Name: class, dtype: int64

## STEP 6: Use the InceptionResnetV3 pre-trained model to create the base model

Lets now re-use InceptionResnetV3 as it records best performance, freeze the base layers and lets add and train the top few layers (https://medium.com/neuralspace/kaggle-1-winning-approach-for-image-classification-challenge-9c1188157a86)

In [23]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = InceptionResNetV2(weights='imagenet',include_top=False) 


 53985280/219055592 [======>.......................] - ETA: 2:03:3 - ETA: 1:22:5 - ETA: 1:13:5 - ETA: 1:10:3 - ETA: 48:09  - ETA: 48:2 - ETA: 32:4 - ETA: 28:5 - ETA: 27:1 - ETA: 19:3 - ETA: 17:5 - ETA: 16:2 - ETA: 12:3 - ETA: 10:2 - ETA: 10:2 - ETA: 9:3 - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA:

214384640/219055592 [============================>.] - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 

219062272/219055592 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 172s 1us/step


## STEP 7: Extract features from this pre-trained model for our training and validation images

In [24]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(2076, 8, 8, 1536)

In [25]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(520, 8, 8, 1536)

In [26]:
shape_X_train = X_train.shape
shape_X_test = X_test.shape

## STEP 8: reshape the images into a single dimension

In [27]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(shape_X_train[0], shape_X_train[1]*shape_X_train[2]*shape_X_train[3])
X_test = X_test.reshape(shape_X_test[0], shape_X_test[1]*shape_X_test[2]*shape_X_test[3])

## STEP 9: Normalize the pixel values (between 0 and 1, helps the model to converge faster)

In [28]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

## STEP 10: Create the architecture of the model

In [29]:
# shape of images
X_train.shape

(2076, 98304)

In [30]:
shape_X_train = X_train.shape

In [31]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(shape_X_train[1],)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## STEP 11: Train our model using the training frames and validate using validation frames

In [32]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight_InceptionResnetV2.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

Improving Generalization Performance by Switching from Adam to SGD
(https://towardsdatascience.com/normalized-direction-preserving-adam-switching-from-adam-to-sgd-and-nesterov-momentum-adam-with-460be5ddf686)

In [33]:
# compiling the model
from keras.optimizers import SGD
opt = SGD()
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              100664320 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
__________

In [35]:
# training the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=64, verbose=1)

Train on 2076 samples, validate on 520 samples
Epoch 1/100
2076/2076 [==============================] - ETA: 9:51 - loss: 0.6964 - acc: 0.476 - ETA: 4:56 - loss: 0.6926 - acc: 0.496 - ETA: 3:18 - loss: 0.6925 - acc: 0.502 - ETA: 2:29 - loss: 0.6935 - acc: 0.505 - ETA: 1:58 - loss: 0.6927 - acc: 0.517 - ETA: 1:38 - loss: 0.6927 - acc: 0.526 - ETA: 1:23 - loss: 0.6908 - acc: 0.535 - ETA: 1:12 - loss: 0.6898 - acc: 0.542 - ETA: 1:03 - loss: 0.6884 - acc: 0.554 - ETA: 55s - loss: 0.6896 - acc: 0.551 - ETA: 49s - loss: 0.6894 - acc: 0.55 - ETA: 44s - loss: 0.6892 - acc: 0.55 - ETA: 39s - loss: 0.6899 - acc: 0.55 - ETA: 35s - loss: 0.6903 - acc: 0.55 - ETA: 32s - loss: 0.6898 - acc: 0.55 - ETA: 29s - loss: 0.6888 - acc: 0.55 - ETA: 26s - loss: 0.6878 - acc: 0.56 - ETA: 24s - loss: 0.6868 - acc: 0.56 - ETA: 21s - loss: 0.6866 - acc: 0.56 - ETA: 19s - loss: 0.6863 - acc: 0.56 - ETA: 17s - loss: 0.6852 - acc: 0.57 - ETA: 16s - loss: 0.6852 - acc: 0.57 - ETA: 14s - loss: 0.6844 - acc: 0.57 - ETA

2076/2076 [==============================] - ETA: 17s - loss: 0.5842 - acc: 0.71 - ETA: 17s - loss: 0.5893 - acc: 0.68 - ETA: 18s - loss: 0.5751 - acc: 0.70 - ETA: 19s - loss: 0.5889 - acc: 0.68 - ETA: 19s - loss: 0.5970 - acc: 0.67 - ETA: 18s - loss: 0.5871 - acc: 0.69 - ETA: 17s - loss: 0.5870 - acc: 0.68 - ETA: 17s - loss: 0.5825 - acc: 0.69 - ETA: 16s - loss: 0.5837 - acc: 0.69 - ETA: 15s - loss: 0.5814 - acc: 0.69 - ETA: 14s - loss: 0.5804 - acc: 0.70 - ETA: 13s - loss: 0.5836 - acc: 0.69 - ETA: 13s - loss: 0.5875 - acc: 0.68 - ETA: 12s - loss: 0.5865 - acc: 0.68 - ETA: 11s - loss: 0.5899 - acc: 0.67 - ETA: 10s - loss: 0.5892 - acc: 0.68 - ETA: 10s - loss: 0.5915 - acc: 0.67 - ETA: 9s - loss: 0.5946 - acc: 0.6736 - ETA: 8s - loss: 0.5927 - acc: 0.678 - ETA: 8s - loss: 0.5900 - acc: 0.682 - ETA: 7s - loss: 0.5924 - acc: 0.681 - ETA: 6s - loss: 0.5920 - acc: 0.682 - ETA: 6s - loss: 0.5911 - acc: 0.683 - ETA: 5s - loss: 0.5902 - acc: 0.683 - ETA: 4s - loss: 0.5912 - acc: 0.682 - ETA:

2076/2076 [==============================] - ETA: 18s - loss: 0.5291 - acc: 0.69 - ETA: 18s - loss: 0.4953 - acc: 0.71 - ETA: 17s - loss: 0.5313 - acc: 0.71 - ETA: 16s - loss: 0.5216 - acc: 0.72 - ETA: 16s - loss: 0.5220 - acc: 0.71 - ETA: 15s - loss: 0.5283 - acc: 0.71 - ETA: 15s - loss: 0.5197 - acc: 0.71 - ETA: 14s - loss: 0.5145 - acc: 0.72 - ETA: 13s - loss: 0.5229 - acc: 0.71 - ETA: 13s - loss: 0.5341 - acc: 0.68 - ETA: 12s - loss: 0.5326 - acc: 0.69 - ETA: 12s - loss: 0.5225 - acc: 0.70 - ETA: 11s - loss: 0.5230 - acc: 0.70 - ETA: 10s - loss: 0.5239 - acc: 0.70 - ETA: 10s - loss: 0.5246 - acc: 0.70 - ETA: 9s - loss: 0.5287 - acc: 0.6978 - ETA: 9s - loss: 0.5343 - acc: 0.691 - ETA: 8s - loss: 0.5344 - acc: 0.690 - ETA: 8s - loss: 0.5340 - acc: 0.689 - ETA: 7s - loss: 0.5346 - acc: 0.692 - ETA: 6s - loss: 0.5358 - acc: 0.691 - ETA: 6s - loss: 0.5315 - acc: 0.696 - ETA: 5s - loss: 0.5326 - acc: 0.695 - ETA: 5s - loss: 0.5324 - acc: 0.695 - ETA: 4s - loss: 0.5342 - acc: 0.693 - ETA:

2076/2076 [==============================] - ETA: 17s - loss: 0.3848 - acc: 0.81 - ETA: 17s - loss: 0.4004 - acc: 0.78 - ETA: 17s - loss: 0.3979 - acc: 0.78 - ETA: 16s - loss: 0.4228 - acc: 0.76 - ETA: 16s - loss: 0.4308 - acc: 0.76 - ETA: 15s - loss: 0.4375 - acc: 0.75 - ETA: 14s - loss: 0.4431 - acc: 0.74 - ETA: 14s - loss: 0.4480 - acc: 0.75 - ETA: 13s - loss: 0.4625 - acc: 0.74 - ETA: 13s - loss: 0.4589 - acc: 0.74 - ETA: 12s - loss: 0.4560 - acc: 0.74 - ETA: 12s - loss: 0.4610 - acc: 0.74 - ETA: 11s - loss: 0.4620 - acc: 0.73 - ETA: 10s - loss: 0.4620 - acc: 0.73 - ETA: 10s - loss: 0.4565 - acc: 0.74 - ETA: 9s - loss: 0.4513 - acc: 0.7495 - ETA: 9s - loss: 0.4494 - acc: 0.749 - ETA: 8s - loss: 0.4486 - acc: 0.749 - ETA: 8s - loss: 0.4482 - acc: 0.750 - ETA: 7s - loss: 0.4470 - acc: 0.752 - ETA: 6s - loss: 0.4484 - acc: 0.749 - ETA: 6s - loss: 0.4495 - acc: 0.746 - ETA: 5s - loss: 0.4524 - acc: 0.742 - ETA: 5s - loss: 0.4537 - acc: 0.742 - ETA: 4s - loss: 0.4570 - acc: 0.739 - ETA:

2076/2076 [==============================] - ETA: 18s - loss: 0.2714 - acc: 0.91 - ETA: 18s - loss: 0.3158 - acc: 0.86 - ETA: 17s - loss: 0.3335 - acc: 0.85 - ETA: 16s - loss: 0.3252 - acc: 0.86 - ETA: 16s - loss: 0.3358 - acc: 0.86 - ETA: 15s - loss: 0.3354 - acc: 0.86 - ETA: 15s - loss: 0.3412 - acc: 0.86 - ETA: 14s - loss: 0.3465 - acc: 0.85 - ETA: 13s - loss: 0.3486 - acc: 0.86 - ETA: 13s - loss: 0.3582 - acc: 0.85 - ETA: 12s - loss: 0.3625 - acc: 0.84 - ETA: 12s - loss: 0.3683 - acc: 0.83 - ETA: 11s - loss: 0.3714 - acc: 0.83 - ETA: 10s - loss: 0.3698 - acc: 0.83 - ETA: 10s - loss: 0.3727 - acc: 0.83 - ETA: 9s - loss: 0.3706 - acc: 0.8350 - ETA: 9s - loss: 0.3753 - acc: 0.829 - ETA: 8s - loss: 0.3740 - acc: 0.830 - ETA: 7s - loss: 0.3748 - acc: 0.832 - ETA: 7s - loss: 0.3727 - acc: 0.835 - ETA: 6s - loss: 0.3717 - acc: 0.835 - ETA: 6s - loss: 0.3680 - acc: 0.838 - ETA: 5s - loss: 0.3680 - acc: 0.837 - ETA: 5s - loss: 0.3677 - acc: 0.838 - ETA: 4s - loss: 0.3664 - acc: 0.838 - ETA:

2076/2076 [==============================] - ETA: 17s - loss: 0.2603 - acc: 0.91 - ETA: 17s - loss: 0.2445 - acc: 0.91 - ETA: 16s - loss: 0.2588 - acc: 0.92 - ETA: 16s - loss: 0.2530 - acc: 0.92 - ETA: 15s - loss: 0.2615 - acc: 0.91 - ETA: 15s - loss: 0.2718 - acc: 0.91 - ETA: 14s - loss: 0.2699 - acc: 0.91 - ETA: 14s - loss: 0.2722 - acc: 0.90 - ETA: 13s - loss: 0.2675 - acc: 0.91 - ETA: 12s - loss: 0.2706 - acc: 0.90 - ETA: 12s - loss: 0.2800 - acc: 0.90 - ETA: 11s - loss: 0.2780 - acc: 0.90 - ETA: 11s - loss: 0.2793 - acc: 0.90 - ETA: 10s - loss: 0.2782 - acc: 0.90 - ETA: 9s - loss: 0.2776 - acc: 0.9073 - ETA: 9s - loss: 0.2768 - acc: 0.906 - ETA: 8s - loss: 0.2774 - acc: 0.904 - ETA: 8s - loss: 0.2783 - acc: 0.902 - ETA: 7s - loss: 0.2755 - acc: 0.905 - ETA: 7s - loss: 0.2737 - acc: 0.907 - ETA: 6s - loss: 0.2704 - acc: 0.908 - ETA: 6s - loss: 0.2712 - acc: 0.908 - ETA: 5s - loss: 0.2738 - acc: 0.905 - ETA: 4s - loss: 0.2733 - acc: 0.906 - ETA: 4s - loss: 0.2719 - acc: 0.907 - ETA:

2076/2076 [==============================] - ETA: 17s - loss: 0.1386 - acc: 0.96 - ETA: 17s - loss: 0.1517 - acc: 0.96 - ETA: 16s - loss: 0.1620 - acc: 0.96 - ETA: 16s - loss: 0.1548 - acc: 0.97 - ETA: 15s - loss: 0.1598 - acc: 0.97 - ETA: 15s - loss: 0.1754 - acc: 0.96 - ETA: 14s - loss: 0.1700 - acc: 0.96 - ETA: 14s - loss: 0.1662 - acc: 0.96 - ETA: 13s - loss: 0.1651 - acc: 0.96 - ETA: 13s - loss: 0.1662 - acc: 0.96 - ETA: 12s - loss: 0.1694 - acc: 0.95 - ETA: 11s - loss: 0.1679 - acc: 0.96 - ETA: 11s - loss: 0.1673 - acc: 0.95 - ETA: 10s - loss: 0.1646 - acc: 0.96 - ETA: 10s - loss: 0.1646 - acc: 0.96 - ETA: 9s - loss: 0.1672 - acc: 0.9600 - ETA: 9s - loss: 0.1658 - acc: 0.960 - ETA: 8s - loss: 0.1702 - acc: 0.959 - ETA: 7s - loss: 0.1680 - acc: 0.961 - ETA: 7s - loss: 0.1721 - acc: 0.957 - ETA: 6s - loss: 0.1745 - acc: 0.955 - ETA: 6s - loss: 0.1721 - acc: 0.957 - ETA: 5s - loss: 0.1713 - acc: 0.958 - ETA: 4s - loss: 0.1707 - acc: 0.959 - ETA: 4s - loss: 0.1703 - acc: 0.959 - ETA:

2076/2076 [==============================] - ETA: 20s - loss: 0.1024 - acc: 0.98 - ETA: 19s - loss: 0.0820 - acc: 0.99 - ETA: 18s - loss: 0.0839 - acc: 0.98 - ETA: 18s - loss: 0.0829 - acc: 0.99 - ETA: 17s - loss: 0.0875 - acc: 0.99 - ETA: 16s - loss: 0.0833 - acc: 0.99 - ETA: 15s - loss: 0.0973 - acc: 0.98 - ETA: 15s - loss: 0.1001 - acc: 0.98 - ETA: 14s - loss: 0.0985 - acc: 0.98 - ETA: 13s - loss: 0.1000 - acc: 0.98 - ETA: 13s - loss: 0.1028 - acc: 0.98 - ETA: 12s - loss: 0.1055 - acc: 0.97 - ETA: 11s - loss: 0.1069 - acc: 0.97 - ETA: 11s - loss: 0.1058 - acc: 0.97 - ETA: 10s - loss: 0.1032 - acc: 0.98 - ETA: 10s - loss: 0.1024 - acc: 0.98 - ETA: 9s - loss: 0.1006 - acc: 0.9821 - ETA: 8s - loss: 0.1015 - acc: 0.982 - ETA: 8s - loss: 0.0999 - acc: 0.983 - ETA: 7s - loss: 0.1022 - acc: 0.982 - ETA: 6s - loss: 0.1073 - acc: 0.979 - ETA: 6s - loss: 0.1079 - acc: 0.978 - ETA: 5s - loss: 0.1081 - acc: 0.978 - ETA: 5s - loss: 0.1103 - acc: 0.976 - ETA: 4s - loss: 0.1098 - acc: 0.976 - ETA:

2076/2076 [==============================] - ETA: 20s - loss: 0.0739 - acc: 0.99 - ETA: 19s - loss: 0.0690 - acc: 0.99 - ETA: 18s - loss: 0.0664 - acc: 0.99 - ETA: 17s - loss: 0.0622 - acc: 0.99 - ETA: 16s - loss: 0.0693 - acc: 0.99 - ETA: 16s - loss: 0.0696 - acc: 0.99 - ETA: 16s - loss: 0.0660 - acc: 0.99 - ETA: 16s - loss: 0.0654 - acc: 0.99 - ETA: 16s - loss: 0.0649 - acc: 0.99 - ETA: 15s - loss: 0.0643 - acc: 0.99 - ETA: 14s - loss: 0.0682 - acc: 0.99 - ETA: 13s - loss: 0.0686 - acc: 0.98 - ETA: 13s - loss: 0.0714 - acc: 0.98 - ETA: 12s - loss: 0.0731 - acc: 0.98 - ETA: 11s - loss: 0.0717 - acc: 0.98 - ETA: 10s - loss: 0.0702 - acc: 0.98 - ETA: 9s - loss: 0.0714 - acc: 0.9867 - ETA: 9s - loss: 0.0707 - acc: 0.986 - ETA: 8s - loss: 0.0711 - acc: 0.985 - ETA: 7s - loss: 0.0700 - acc: 0.986 - ETA: 7s - loss: 0.0727 - acc: 0.985 - ETA: 6s - loss: 0.0765 - acc: 0.984 - ETA: 5s - loss: 0.0752 - acc: 0.985 - ETA: 5s - loss: 0.0760 - acc: 0.984 - ETA: 4s - loss: 0.0755 - acc: 0.984 - ETA:

2076/2076 [==============================] - ETA: 18s - loss: 0.0528 - acc: 1.00 - ETA: 18s - loss: 0.0452 - acc: 1.00 - ETA: 18s - loss: 0.0461 - acc: 1.00 - ETA: 18s - loss: 0.0442 - acc: 1.00 - ETA: 17s - loss: 0.0423 - acc: 1.00 - ETA: 16s - loss: 0.0417 - acc: 1.00 - ETA: 15s - loss: 0.0434 - acc: 1.00 - ETA: 15s - loss: 0.0429 - acc: 1.00 - ETA: 14s - loss: 0.0409 - acc: 1.00 - ETA: 13s - loss: 0.0402 - acc: 1.00 - ETA: 13s - loss: 0.0415 - acc: 0.99 - ETA: 12s - loss: 0.0419 - acc: 0.99 - ETA: 11s - loss: 0.0422 - acc: 0.99 - ETA: 11s - loss: 0.0420 - acc: 0.99 - ETA: 10s - loss: 0.0414 - acc: 0.99 - ETA: 10s - loss: 0.0443 - acc: 0.99 - ETA: 9s - loss: 0.0478 - acc: 0.9945 - ETA: 8s - loss: 0.0510 - acc: 0.991 - ETA: 8s - loss: 0.0511 - acc: 0.991 - ETA: 7s - loss: 0.0513 - acc: 0.991 - ETA: 6s - loss: 0.0504 - acc: 0.991 - ETA: 6s - loss: 0.0502 - acc: 0.991 - ETA: 5s - loss: 0.0502 - acc: 0.991 - ETA: 5s - loss: 0.0495 - acc: 0.992 - ETA: 4s - loss: 0.0489 - acc: 0.992 - ETA:

2076/2076 [==============================] - ETA: 19s - loss: 0.0249 - acc: 1.00 - ETA: 18s - loss: 0.0186 - acc: 1.00 - ETA: 17s - loss: 0.0182 - acc: 1.00 - ETA: 17s - loss: 0.0194 - acc: 1.00 - ETA: 16s - loss: 0.0242 - acc: 0.99 - ETA: 15s - loss: 0.0282 - acc: 0.99 - ETA: 15s - loss: 0.0283 - acc: 0.99 - ETA: 14s - loss: 0.0289 - acc: 0.99 - ETA: 14s - loss: 0.0287 - acc: 0.99 - ETA: 13s - loss: 0.0288 - acc: 0.99 - ETA: 12s - loss: 0.0292 - acc: 0.99 - ETA: 12s - loss: 0.0291 - acc: 0.99 - ETA: 11s - loss: 0.0289 - acc: 0.99 - ETA: 11s - loss: 0.0310 - acc: 0.99 - ETA: 10s - loss: 0.0315 - acc: 0.99 - ETA: 10s - loss: 0.0321 - acc: 0.99 - ETA: 9s - loss: 0.0319 - acc: 0.9945 - ETA: 8s - loss: 0.0313 - acc: 0.994 - ETA: 8s - loss: 0.0316 - acc: 0.995 - ETA: 7s - loss: 0.0334 - acc: 0.994 - ETA: 7s - loss: 0.0323 - acc: 0.994 - ETA: 6s - loss: 0.0320 - acc: 0.995 - ETA: 5s - loss: 0.0325 - acc: 0.994 - ETA: 5s - loss: 0.0318 - acc: 0.994 - ETA: 4s - loss: 0.0318 - acc: 0.995 - ETA:

2076/2076 [==============================] - ETA: 17s - loss: 0.0520 - acc: 0.97 - ETA: 18s - loss: 0.0369 - acc: 0.98 - ETA: 18s - loss: 0.0292 - acc: 0.99 - ETA: 17s - loss: 0.0307 - acc: 0.99 - ETA: 18s - loss: 0.0296 - acc: 0.99 - ETA: 20s - loss: 0.0264 - acc: 0.99 - ETA: 19s - loss: 0.0280 - acc: 0.99 - ETA: 17s - loss: 0.0274 - acc: 0.99 - ETA: 16s - loss: 0.0261 - acc: 0.99 - ETA: 15s - loss: 0.0262 - acc: 0.99 - ETA: 14s - loss: 0.0257 - acc: 0.99 - ETA: 13s - loss: 0.0256 - acc: 0.99 - ETA: 13s - loss: 0.0261 - acc: 0.99 - ETA: 12s - loss: 0.0256 - acc: 0.99 - ETA: 11s - loss: 0.0246 - acc: 0.99 - ETA: 11s - loss: 0.0237 - acc: 0.99 - ETA: 10s - loss: 0.0231 - acc: 0.99 - ETA: 9s - loss: 0.0231 - acc: 0.9983 - ETA: 8s - loss: 0.0226 - acc: 0.998 - ETA: 8s - loss: 0.0221 - acc: 0.998 - ETA: 7s - loss: 0.0218 - acc: 0.998 - ETA: 6s - loss: 0.0216 - acc: 0.998 - ETA: 6s - loss: 0.0220 - acc: 0.998 - ETA: 5s - loss: 0.0220 - acc: 0.998 - ETA: 4s - loss: 0.0226 - acc: 0.997 - ETA:

2076/2076 [==============================] - ETA: 22s - loss: 0.0133 - acc: 1.00 - ETA: 24s - loss: 0.0152 - acc: 1.00 - ETA: 22s - loss: 0.0144 - acc: 1.00 - ETA: 21s - loss: 0.0213 - acc: 0.99 - ETA: 22s - loss: 0.0200 - acc: 0.99 - ETA: 20s - loss: 0.0199 - acc: 0.99 - ETA: 19s - loss: 0.0192 - acc: 0.99 - ETA: 19s - loss: 0.0211 - acc: 0.99 - ETA: 18s - loss: 0.0225 - acc: 0.99 - ETA: 17s - loss: 0.0237 - acc: 0.99 - ETA: 16s - loss: 0.0223 - acc: 0.99 - ETA: 16s - loss: 0.0224 - acc: 0.99 - ETA: 15s - loss: 0.0226 - acc: 0.99 - ETA: 14s - loss: 0.0219 - acc: 0.99 - ETA: 13s - loss: 0.0214 - acc: 0.99 - ETA: 13s - loss: 0.0213 - acc: 0.99 - ETA: 12s - loss: 0.0207 - acc: 0.99 - ETA: 11s - loss: 0.0211 - acc: 0.99 - ETA: 10s - loss: 0.0213 - acc: 0.99 - ETA: 9s - loss: 0.0210 - acc: 0.9969 - ETA: 9s - loss: 0.0208 - acc: 0.997 - ETA: 8s - loss: 0.0203 - acc: 0.997 - ETA: 7s - loss: 0.0205 - acc: 0.997 - ETA: 6s - loss: 0.0204 - acc: 0.997 - ETA: 5s - loss: 0.0203 - acc: 0.997 - ETA:

2076/2076 [==============================] - ETA: 20s - loss: 0.0113 - acc: 1.00 - ETA: 18s - loss: 0.0126 - acc: 1.00 - ETA: 18s - loss: 0.0191 - acc: 0.99 - ETA: 17s - loss: 0.0173 - acc: 0.99 - ETA: 17s - loss: 0.0158 - acc: 0.99 - ETA: 17s - loss: 0.0159 - acc: 0.99 - ETA: 17s - loss: 0.0153 - acc: 0.99 - ETA: 16s - loss: 0.0147 - acc: 0.99 - ETA: 15s - loss: 0.0144 - acc: 0.99 - ETA: 14s - loss: 0.0147 - acc: 0.99 - ETA: 14s - loss: 0.0162 - acc: 0.99 - ETA: 13s - loss: 0.0168 - acc: 0.99 - ETA: 12s - loss: 0.0169 - acc: 0.99 - ETA: 11s - loss: 0.0171 - acc: 0.99 - ETA: 11s - loss: 0.0167 - acc: 0.99 - ETA: 10s - loss: 0.0161 - acc: 0.99 - ETA: 9s - loss: 0.0162 - acc: 0.9986 - ETA: 9s - loss: 0.0161 - acc: 0.998 - ETA: 8s - loss: 0.0161 - acc: 0.998 - ETA: 7s - loss: 0.0161 - acc: 0.998 - ETA: 7s - loss: 0.0160 - acc: 0.998 - ETA: 6s - loss: 0.0157 - acc: 0.998 - ETA: 6s - loss: 0.0157 - acc: 0.999 - ETA: 5s - loss: 0.0155 - acc: 0.999 - ETA: 5s - loss: 0.0154 - acc: 0.999 - ETA:

2076/2076 [==============================] - ETA: 21s - loss: 0.0077 - acc: 1.00 - ETA: 20s - loss: 0.0127 - acc: 0.99 - ETA: 19s - loss: 0.0186 - acc: 0.99 - ETA: 19s - loss: 0.0184 - acc: 0.99 - ETA: 18s - loss: 0.0169 - acc: 0.99 - ETA: 17s - loss: 0.0158 - acc: 0.99 - ETA: 16s - loss: 0.0162 - acc: 0.99 - ETA: 16s - loss: 0.0149 - acc: 0.99 - ETA: 15s - loss: 0.0144 - acc: 0.99 - ETA: 14s - loss: 0.0143 - acc: 0.99 - ETA: 14s - loss: 0.0143 - acc: 0.99 - ETA: 13s - loss: 0.0146 - acc: 0.99 - ETA: 12s - loss: 0.0151 - acc: 0.99 - ETA: 12s - loss: 0.0150 - acc: 0.99 - ETA: 11s - loss: 0.0145 - acc: 0.99 - ETA: 10s - loss: 0.0141 - acc: 0.99 - ETA: 10s - loss: 0.0138 - acc: 0.99 - ETA: 9s - loss: 0.0134 - acc: 0.9987 - ETA: 8s - loss: 0.0143 - acc: 0.997 - ETA: 8s - loss: 0.0148 - acc: 0.998 - ETA: 7s - loss: 0.0151 - acc: 0.998 - ETA: 6s - loss: 0.0153 - acc: 0.998 - ETA: 6s - loss: 0.0149 - acc: 0.998 - ETA: 5s - loss: 0.0149 - acc: 0.998 - ETA: 4s - loss: 0.0148 - acc: 0.998 - ETA:

2076/2076 [==============================] - ETA: 20s - loss: 0.0133 - acc: 1.00 - ETA: 19s - loss: 0.0139 - acc: 1.00 - ETA: 20s - loss: 0.0139 - acc: 1.00 - ETA: 19s - loss: 0.0134 - acc: 1.00 - ETA: 18s - loss: 0.0119 - acc: 1.00 - ETA: 18s - loss: 0.0119 - acc: 1.00 - ETA: 17s - loss: 0.0120 - acc: 1.00 - ETA: 17s - loss: 0.0129 - acc: 0.99 - ETA: 16s - loss: 0.0127 - acc: 0.99 - ETA: 16s - loss: 0.0125 - acc: 0.99 - ETA: 15s - loss: 0.0119 - acc: 0.99 - ETA: 14s - loss: 0.0115 - acc: 0.99 - ETA: 13s - loss: 0.0112 - acc: 0.99 - ETA: 13s - loss: 0.0115 - acc: 0.99 - ETA: 13s - loss: 0.0116 - acc: 0.99 - ETA: 12s - loss: 0.0112 - acc: 0.99 - ETA: 11s - loss: 0.0109 - acc: 0.99 - ETA: 11s - loss: 0.0111 - acc: 0.99 - ETA: 10s - loss: 0.0112 - acc: 0.99 - ETA: 9s - loss: 0.0111 - acc: 0.9996 - ETA: 8s - loss: 0.0115 - acc: 0.999 - ETA: 7s - loss: 0.0115 - acc: 0.999 - ETA: 6s - loss: 0.0112 - acc: 0.999 - ETA: 6s - loss: 0.0110 - acc: 0.999 - ETA: 5s - loss: 0.0110 - acc: 0.999 - ETA:

2076/2076 [==============================] - ETA: 19s - loss: 0.0057 - acc: 1.00 - ETA: 18s - loss: 0.0061 - acc: 1.00 - ETA: 17s - loss: 0.0059 - acc: 1.00 - ETA: 16s - loss: 0.0054 - acc: 1.00 - ETA: 15s - loss: 0.0054 - acc: 1.00 - ETA: 15s - loss: 0.0051 - acc: 1.00 - ETA: 14s - loss: 0.0054 - acc: 1.00 - ETA: 14s - loss: 0.0066 - acc: 1.00 - ETA: 13s - loss: 0.0063 - acc: 1.00 - ETA: 13s - loss: 0.0063 - acc: 1.00 - ETA: 12s - loss: 0.0063 - acc: 1.00 - ETA: 12s - loss: 0.0062 - acc: 1.00 - ETA: 11s - loss: 0.0061 - acc: 1.00 - ETA: 11s - loss: 0.0066 - acc: 1.00 - ETA: 10s - loss: 0.0080 - acc: 0.99 - ETA: 9s - loss: 0.0082 - acc: 0.9990 - ETA: 9s - loss: 0.0082 - acc: 0.999 - ETA: 8s - loss: 0.0079 - acc: 0.999 - ETA: 8s - loss: 0.0080 - acc: 0.999 - ETA: 7s - loss: 0.0079 - acc: 0.999 - ETA: 6s - loss: 0.0079 - acc: 0.999 - ETA: 6s - loss: 0.0078 - acc: 0.999 - ETA: 5s - loss: 0.0079 - acc: 0.999 - ETA: 5s - loss: 0.0080 - acc: 0.999 - ETA: 4s - loss: 0.0078 - acc: 0.999 - ETA:

2076/2076 [==============================] - ETA: 18s - loss: 0.0054 - acc: 1.00 - ETA: 20s - loss: 0.0047 - acc: 1.00 - ETA: 20s - loss: 0.0075 - acc: 1.00 - ETA: 19s - loss: 0.0079 - acc: 1.00 - ETA: 18s - loss: 0.0073 - acc: 1.00 - ETA: 17s - loss: 0.0073 - acc: 1.00 - ETA: 16s - loss: 0.0075 - acc: 1.00 - ETA: 15s - loss: 0.0080 - acc: 1.00 - ETA: 15s - loss: 0.0075 - acc: 1.00 - ETA: 14s - loss: 0.0071 - acc: 1.00 - ETA: 13s - loss: 0.0074 - acc: 1.00 - ETA: 13s - loss: 0.0071 - acc: 1.00 - ETA: 12s - loss: 0.0069 - acc: 1.00 - ETA: 11s - loss: 0.0066 - acc: 1.00 - ETA: 10s - loss: 0.0066 - acc: 1.00 - ETA: 10s - loss: 0.0065 - acc: 1.00 - ETA: 9s - loss: 0.0064 - acc: 1.0000 - ETA: 8s - loss: 0.0069 - acc: 1.000 - ETA: 8s - loss: 0.0071 - acc: 1.000 - ETA: 7s - loss: 0.0070 - acc: 1.000 - ETA: 7s - loss: 0.0076 - acc: 1.000 - ETA: 6s - loss: 0.0075 - acc: 1.000 - ETA: 5s - loss: 0.0074 - acc: 1.000 - ETA: 5s - loss: 0.0077 - acc: 0.999 - ETA: 4s - loss: 0.0075 - acc: 0.999 - ETA:

2076/2076 [==============================] - ETA: 18s - loss: 0.0054 - acc: 1.00 - ETA: 17s - loss: 0.0068 - acc: 1.00 - ETA: 17s - loss: 0.0059 - acc: 1.00 - ETA: 16s - loss: 0.0050 - acc: 1.00 - ETA: 15s - loss: 0.0057 - acc: 1.00 - ETA: 15s - loss: 0.0064 - acc: 1.00 - ETA: 14s - loss: 0.0065 - acc: 1.00 - ETA: 14s - loss: 0.0089 - acc: 0.99 - ETA: 14s - loss: 0.0086 - acc: 0.99 - ETA: 13s - loss: 0.0084 - acc: 0.99 - ETA: 13s - loss: 0.0086 - acc: 0.99 - ETA: 12s - loss: 0.0082 - acc: 0.99 - ETA: 12s - loss: 0.0079 - acc: 0.99 - ETA: 11s - loss: 0.0077 - acc: 0.99 - ETA: 10s - loss: 0.0075 - acc: 0.99 - ETA: 10s - loss: 0.0073 - acc: 0.99 - ETA: 9s - loss: 0.0072 - acc: 0.9991 - ETA: 8s - loss: 0.0071 - acc: 0.999 - ETA: 8s - loss: 0.0068 - acc: 0.999 - ETA: 7s - loss: 0.0066 - acc: 0.999 - ETA: 7s - loss: 0.0067 - acc: 0.999 - ETA: 6s - loss: 0.0067 - acc: 0.999 - ETA: 5s - loss: 0.0068 - acc: 0.999 - ETA: 5s - loss: 0.0066 - acc: 0.999 - ETA: 4s - loss: 0.0071 - acc: 0.999 - ETA:

2076/2076 [==============================] - ETA: 18s - loss: 0.0043 - acc: 1.00 - ETA: 17s - loss: 0.0042 - acc: 1.00 - ETA: 17s - loss: 0.0042 - acc: 1.00 - ETA: 17s - loss: 0.0040 - acc: 1.00 - ETA: 17s - loss: 0.0041 - acc: 1.00 - ETA: 16s - loss: 0.0050 - acc: 1.00 - ETA: 15s - loss: 0.0052 - acc: 1.00 - ETA: 14s - loss: 0.0051 - acc: 1.00 - ETA: 14s - loss: 0.0050 - acc: 1.00 - ETA: 13s - loss: 0.0052 - acc: 1.00 - ETA: 12s - loss: 0.0051 - acc: 1.00 - ETA: 12s - loss: 0.0049 - acc: 1.00 - ETA: 11s - loss: 0.0050 - acc: 1.00 - ETA: 10s - loss: 0.0048 - acc: 1.00 - ETA: 10s - loss: 0.0048 - acc: 1.00 - ETA: 10s - loss: 0.0048 - acc: 1.00 - ETA: 9s - loss: 0.0048 - acc: 1.0000 - ETA: 9s - loss: 0.0056 - acc: 0.999 - ETA: 8s - loss: 0.0055 - acc: 0.999 - ETA: 7s - loss: 0.0057 - acc: 0.999 - ETA: 7s - loss: 0.0058 - acc: 0.999 - ETA: 6s - loss: 0.0059 - acc: 0.999 - ETA: 5s - loss: 0.0058 - acc: 0.999 - ETA: 5s - loss: 0.0057 - acc: 0.999 - ETA: 4s - loss: 0.0058 - acc: 0.999 - ETA:

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


In [36]:
import matplotlib.pyplot as plt

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()